In [1]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader 


In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Domain specific libraries 
# Torchtext, torchvision, torchaudio etc 
# all of them include datasets

In [4]:
# Download training data from the open datasets 
training_data = datasets.FashionMNIST(
    root = "data", 
    train = True, 
    download = True, 
    transform=ToTensor() 
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
# Download the test data from the open datasets 
test_data = datasets.FashionMNIST(
    root = "data", 
    train = False, 
    download = True, 
    transform=ToTensor() 
)

In [6]:
# Pass Dataset as an argument to the DataLoader 
# This wraps the iterable over dataset 
# supports automatic batching, sampling, shuffling and multiprocess data loading 

In [7]:
# Here we define a batch size of 64 
# each element in the data loader iterable will return a batch of 64 features and labels 

In [8]:
batch_size = 64 

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size) 
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# iterate over the test data loader 
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") 
    print(f"Shape of y: {y.shape}, dtype: {y.dtype}") 
    break 

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), dtype: torch.int64


In [9]:
# LOADING DATA IN PYTORCH ?? READ MORE 

In [10]:
# CREATE MODELS 
# DEFINE A NUERAL NETWORK 
# CREATE A CLASS THAT INHERITS FROM NN.MODULE 
# DEFINE THE LAYERS IN THE __INIT__ METHOD 
# SPECIFY HOW DATA WILL PASS THROUGH THE NETWORK IN THE FORWARD FUNCTION 


In [12]:
device = "cuda" if torch.cuda.is_available() else "mps"  if torch.backends.mps.is_available() else "cpu" 
print("Using {} device".format(device))

Using mps device


In [14]:
# DEFINE THE MODEL 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() 
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10), 
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x) 
        logits = self.linear_relu_stack(x) 
        return logits
    
model = NeuralNetwork().to(device) 
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [15]:
# OPTIMISE THE MODELS PARAMETERS 
# Loss function and an Optimiser 
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 

In [16]:
# Single training loop 
# model makes a prediction on the training dataset(fed into it in batches) 
# backpropagates the prediction error to adjust the models parameters 

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) 
    model.train() 
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) 

        # Compute prediction error 
        pred = model(X) 
        loss = loss_fn(pred, y) 

        # Backpropagation 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) 
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
# Check models performance against the test dataset to ensure it is learning 

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) 
    num_batches = len(dataloader) 
    model.eval() 
    test_loss, correct = 0, 0 
    with torch.no_grad(): 
        for X, y in dataloader: 
            X, y = X.to(device), y.to(device) 
            pred = model(X) 
            test_loss += loss_fn(pred, y).item() 
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() 
    test_loss /= num_batches 
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
# training is conducted over several iterations(epochs) 
# during each epoch, the model learns the parameters to make better predictions 
# print the models accuracy and loss at each epoch 
# like to see the accuracy increase and the loss decrease with each epoch 

epochs = 5 
for t in range(epochs): 
    print(f"Epoch {t+1}\n-------------------------------") 
    train(train_dataloader, model, loss_fn, optimizer) 
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.208367  [    0/60000]
loss: 2.205538  [ 6400/60000]
loss: 2.175429  [12800/60000]
loss: 2.186176  [19200/60000]
loss: 2.108014  [25600/60000]
loss: 2.104256  [32000/60000]
loss: 2.122432  [38400/60000]
loss: 2.081427  [44800/60000]
loss: 2.058642  [51200/60000]
loss: 2.028870  [57600/60000]
Test Error: 
 Accuracy: 44.2%, Avg loss: 2.053208 

Epoch 2
-------------------------------
loss: 2.069063  [    0/60000]
loss: 2.060877  [ 6400/60000]
loss: 2.005819  [12800/60000]
loss: 2.037919  [19200/60000]
loss: 1.894453  [25600/60000]
loss: 1.906089  [32000/60000]
loss: 1.929974  [38400/60000]
loss: 1.873700  [44800/60000]
loss: 1.852670  [51200/60000]
loss: 1.806255  [57600/60000]
Test Error: 
 Accuracy: 45.9%, Avg loss: 1.834801 

Epoch 3
-------------------------------
loss: 1.891272  [    0/60000]
loss: 1.844258  [ 6400/60000]
loss: 1.762447  [12800/60000]
loss: 1.793318  [19200/60000]
loss: 1.626122  [25600/60000]
loss: 1.698185  [32000/600

In [21]:
# SAVE THE MODEL 
torch.save(model.state_dict(), "model.pth") 
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [22]:
# LOAD THE MODEL 
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [23]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
